<a href="https://colab.research.google.com/drive/1_E7kjUJ6IU_NL2jzGjMIBx2dz4q_nvcz?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to GPU Programming with CuPy
## Estimating Pi with Monte Carlo Simulation

In this notebook, we'll walk through how we can use the `cupy` package -- with friendly `numpy`-style syntax -- to accelerate our embarrasingly parallel code using GPUs. Our goal today will be to a estimate a value -- $\pi$ -- [using Monte Carlo Simulation](https://en.wikipedia.org/wiki/Monte_Carlo_method#Overview) (a classic task for demonstrating HPC speedups and applicable to any numerical estimation task in the social sciences).

First taking a look at the hardware we have available to work with:

In [1]:
!nvidia-smi

Fri Mar 29 23:34:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Serially, we can solve this problem in `numpy` like so (generating 100k coordinates in a unit square and identifying whether they fall in a unit circle or not):

In [57]:
# NumPy Pi Estimation with Monte Carlo Simulation
import numpy as np
import time

n_runs = 10 ** 5

In [58]:
%%timeit
# Simulate Random Coordinates in Unit Square:
ran = np.random.uniform(low=-1, high=1, size=(2, n_runs))

# Identify Random Coordinates that fall within Unit Circle and count them
result = ran[0] ** 2 + ran[1] ** 2 <= 1
n_in_circle = np.sum(result)

# Compute pi
pi = 4 * n_in_circle / n_runs

2.18 ms ± 41.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## GPU Programming with `CuPy`

A useful approach for working with array data on GPUs is to use `CuPy`, which replicates much of the functionality of `numpy`, but on GPUs. You can take a look at [the documentation](https://docs.cupy.dev/en/stable/user_guide/basic.html) for more detail on all of the GPU functions that are available in CuPy. If you need to write a function that is not supported in CuPy, you can either [define a your own GPU kernel](https://docs.cupy.dev/en/stable/user_guide/kernel.html) (beyond the scope of the class, as this requires some exposure to underlying CUDA syntax). Note that [CuPy arrays are also communicable via mpi4py](https://docs.cupy.dev/en/stable/user_guide/interoperability.html#mpi4py) in a distributed GPU context. Let's import CuPy:


In [12]:
import cupy as cp

To use CuPy on our GPU, we just need to copy our NumPy arrays over to our GPU as CuPy arrays. Then, we can perform array operations as if we working with NumPy as usual, but accelerated on our GPU. In order to work with the data again on our CPU, we will need to copy it back from the GPU Device to our CPU host. For instance, to compute pi, we might write the following:

In [65]:
%%timeit
ran = np.random.uniform(low=-1, high =1, size=(2, n_runs)).astype(np.float64)
x, y = ran[0], ran[1]

# copy x and y to GPU Device
x_dev, y_dev = cp.asarray(x), cp.asarray(y)

# Perform Computation on GPU Device
result_dev = x_dev ** 2 + y_dev ** 2 <= 1

# Copy result from GPU Device to CPU Host
result = result_dev.get() # alternatively: cp.asnumpy(result_dev)

# Perform final computations on CPU Host
n_in_circle = np.sum(result)
pi = 4 * n_in_circle / n_runs

3.02 ms ± 108 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


This is actually slower than our standard NumPy CPU solution, though. Why?

## Copying data between CPU Host and GPU Device

One of the biggest bottlenecks when working with GPUs is the time it takes to copy data back and forth between the CPU Host and GPU Device. We can try to minimize this time, though, by running more operations on the GPU in an attempt to send and receive smaller communications from CPU-to-GPU and GPU-to-CPU.

For instance, in this application, we can perform the summation on the GPU...

In [66]:
%%timeit
ran = np.random.uniform(low=-1, high =1, size=(2, n_runs)).astype(np.float64)
x, y = ran[0], ran[1]

# copy x and y to GPU Device
x_dev, y_dev = cp.asarray(x), cp.asarray(y)

# Perform Computation on GPU Device
result_dev = x_dev ** 2 + y_dev ** 2 <= 1

# Perform final computations on GPU Host
n_in_circle_dev = cp.sum(result_dev)
pi_dev = 4 * n_in_circle_dev / n_runs

# Copy result from GPU Device to CPU Host
pi = pi_dev.get()

2.87 ms ± 60.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


We can also generate random numbers on our GPU in exactly the same way we did in `numpy` (but our data is now on our GPU and doesn't need to be communicated over):



In [67]:
%%time
x_dev, y_dev = cp.random.uniform(low=-1, high=1, size=(2, n_runs), dtype=np.float64)

# Perform Computation on GPU Device
result_dev = x_dev ** 2 + y_dev ** 2 <= 1

# Perform final computations on GPU Host
n_in_circle_dev = cp.sum(result_dev)
pi_dev = 4 * n_in_circle_dev / n_runs

# Copy result from GPU Device to CPU Host
pi = pi_dev.get()

CPU times: user 1.15 ms, sys: 21 µs, total: 1.17 ms
Wall time: 1.18 ms


## Making it worth the cost to send to a GPU: Data Size and Arithmetic Complexity of Parallel Task

GPUs are also best served when the tasks they are performing are on large data sizes, or of especially high arithmetic complexity. Otherwise, the cost of sending/receiving data from GPU-to-CPU may not be worth it.

Let's increase the number of runs we are simulating here to 100 million and see how our CPU and GPU solutions compare now:

In [68]:
n_runs = 10 ** 8

In [69]:
%%timeit
# Simulate Random Coordinates in Unit Square:
ran = np.random.uniform(low=-1, high=1, size=(2, n_runs))

# Identify Random Coordinates that fall within Unit Circle and count them
result = ran[0] ** 2 + ran[1] ** 2 <= 1
n_in_circle = np.sum(result)

# Compute pi
pi = 4 * n_in_circle / n_runs

3.28 s ± 388 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [70]:
%%timeit
x_dev, y_dev = cp.random.uniform(low=-1, high=1, size=(2, n_runs), dtype=np.float64)

# Perform Computation on GPU Device
result_dev = x_dev ** 2 + y_dev ** 2 <= 1

# Perform final computations on GPU Host
n_in_circle_dev = cp.sum(result_dev)
pi_dev = 4 * n_in_circle_dev / n_runs

# Copy result from GPU Device to CPU Host
pi = pi_dev.get()

214 ms ± 771 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Higher Precision than needed?

Sometimes, the precision of our datatypes is higher than it needs to be. For instance, we've been performing simulations with double precision floats, but may not need this high of precision for our application. Let's see the impact of lowering our precision:

In [71]:
%%timeit
x_dev, y_dev = cp.random.uniform(low=-1, high=1, size=(2, n_runs), dtype=np.float32)

# copy x and y to GPU Device
x_dev, y_dev = cp.asarray(x), cp.asarray(y)

# Perform Computation on GPU Device
result_dev = x_dev ** 2 + y_dev ** 2 <= 1

# Perform final computations on GPU Host
n_in_circle_dev = cp.sum(result_dev)
pi_dev = 4 * n_in_circle_dev / n_runs

# Copy result from GPU Device to CPU Host
pi = pi_dev.get()

17.8 ms ± 12.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Benchmarking GPU vs. CPU Usage

CuPy also conveniently includes a benchmarking function that explicitly times how much work is being done on GPU versus CPU. We can see, for instance, that if we run the same code and `.get()` or result back on our CPU, there is a significant amount of time that is spent on CPU operations (reinforcing the need to minimize data transfer between CPU and GPU, as well as maximize the work that is done on the GPU):

In [55]:
from cupyx.profiler import benchmark

def cupy_pi_est(x, y):
  # Compute on GPU via CuPy arrays: ~175k us (microseconds)
  result = x ** 2 + y ** 2 <= 1
  n_in_circle = cp.sum(result)
  pi = 4 * n_in_circle / n_runs

  # Get pi back to host CPU from GPU: ~175k us (microseconds)
  # half of time is spent just getting pi back to host CPU from GPU device
  pi_cpu = pi.get()

  return pi_cpu

benchmark(cupy_pi_est, (x_dev, y_dev), n_repeat=10)

cupy_pi_est         :    CPU: 175379.738 us   +/- 859.394 (min: 174080.136 / max: 177087.083) us     GPU-0: 175424.187 us   +/- 858.840 (min: 174122.589 / max: 177125.946) us

Whereas if we perform the same operation without bringing the estimated value back to the CPU via `.get()`, our GPU time remains the same, but there is quite a bit less time spent overall (since there is no data transfer time):

In [56]:
def cupy_pi_est(x, y):
  # Compute on GPU via CuPy arrays: ~175k us (microseconds)
  result = x ** 2 + y ** 2 <= 1
  n_in_circle = cp.sum(result)
  pi = 4 * n_in_circle / n_runs

  # don't send result back to CPU here for timing purposes
  return pi

benchmark(cupy_pi_est, (x_dev, y_dev), n_repeat=10)

cupy_pi_est         :    CPU:   588.843 us   +/- 151.390 (min:   498.381 / max:   999.069) us     GPU-0: 176693.759 us   +/- 1237.829 (min: 174334.595 / max: 178280.441) us